# Reproduction of Tangri et al.'s Kidney Failure Risk Equation (KFRE)

## Import Libraries

In [1]:
########################### Import Requisite Libraries #########################
import pandas as pd
import numpy as np
import os
import sys
import pprint

################################################################################
# Add the parent directory to sys.path to access 'functions.py'
sys.path.append(os.path.join(os.pardir))
from python_scripts.functions import *
from python_scripts.kfre import *

## Read in the Data

In [2]:
base_path = os.path.join(os.pardir)

# Go up one level from 'notebooks' to the parent directory, then into the 'data' folder
data_path = os.path.join(os.pardir, "data")

image_path_png = os.path.join(base_path, "images", "png_images")
image_path_svg = os.path.join(base_path, "images", "svg_images")

# Use the function to ensure the 'data' directory exists
ensure_directory(data_path)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

# Read the CSV file into a DataFrame
df = pd.read_parquet(os.path.join(data_path, "df.parquet"))
df_eda = pd.read_parquet(os.path.join(data_path, "df_eda.parquet"))

Directory exists: ..\data
Directory exists: ..\images\png_images
Directory exists: ..\images\svg_images


In [3]:
# join back sex to original dataframe for eda dataframe so it can be used
df = df.join(df_eda["SEX"], on="Patient_ID", how="inner")

In [4]:
df_kfre = df.copy(deep=True)

In [5]:
df_kfre.columns

Index(['Age', 'Diabetes (1=yes; 0=no)', 'Hypertension (1=yes; 0=no)',
       'Renal disease (DM=1, HTN=2, GN=3, ADPKD=4, Other=5)', 'eGFR-EPI',
       'uPCR', 'Calcium (mmol/L)', 'Phosphate (mmol/L)',
       'Bicarbonate (mmol/L)', 'Albumin (g/l)', 'Follow-up YEARS', 'RIP',
       'ESRD', 'SEX_Female', 'SEX_Male',
       'ETHNICITY_AFRICAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_ANY OTHER ASIAN BACKGROUND',
       'ETHNICITY_ANY OTHER BLACK BACKGROUND',
       'ETHNICITY_ANY OTHER ETHNIC GROUP',
       'ETHNICITY_ANY OTHER MIXED BACKGROUND',
       'ETHNICITY_ANY OTHER WHITE BACKGROUND',
       'ETHNICITY_BANGLADESHI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_CARIBBEAN (BLACK OR BLACK BRITISH)',
       'ETHNICITY_CHINESE (OTHER ETHNIC GROUPS)',
       'ETHNICITY_INDIAN (ASIAN OR ASIAN BRITISH)', 'ETHNICITY_NOT STATED',
       'ETHNICITY_PAKISTANI (ASIAN OR ASIAN BRITISH)',
       'ETHNICITY_WHITE AND BLACK AFRICAN (MIXED)', 'ETHNICITY_WHITE BRITISH',
       'ETHNICITY_WHITE IRIS

In [6]:
[col for col in df.columns if "Bicarb" in col]

['Bicarbonate (mmol/L)']

In [7]:
df["Bicarbonate (mmol/L)"].median()

21.8

In [8]:
# subset the dataframe to include only relevant variables for the KFRE equation
# Age, eGFR, log-normalized uACR, Diabetes, Hypertension
df_kfre = df_kfre[
    [
        "Age",
        "SEX",
        "eGFR-EPI",
        "uACR",
        "Diabetes (1=yes; 0=no)",
        "Hypertension (1=yes; 0=no)",
        "Albumin (g/dL)",
        "Calcium (mmol/L)",
        "Phosphate (mmol/L)",
        "Bicarbonate (mmol/L)",
        "ESRD",
    ]
]

In [9]:
df_kfre["Region"] = "North American"

In [10]:
columns = {
    "age": "Age",
    "sex": "SEX",
    "eGFR": "eGFR-EPI",
    "uACR": "uACR",
    "region": "Region",
    "dm": "Diabetes (1=yes; 0=no)",
    "htn": "Hypertension (1=yes; 0=no)",
    "albumin": "Albumin (g/dL)",
    "phosphorous": "Phosphate (mmol/L)",
    "bicarbonate": "Bicarbonate (mmol/L)",
    "calcium": "Calcium (mmol/L)",
}

# Assuming `df` is your DataFrame loaded with the dataset
predictor = RiskPredictor(data=df_kfre, columns=columns)
risk = predictor.predict_kfre(2, use_extra_vars=True)


# Predict risk for different combinations of years and variables
pred_4var_2year = predictor.predict_kfre(years=2)
pred_4var_5year = predictor.predict_kfre(years=5)
pred_6var_2year = predictor.predict_kfre(years=2, use_extra_vars=True, num_vars=6)
pred_6var_5year = predictor.predict_kfre(years=5, use_extra_vars=True, num_vars=6)

pred_8var_2year = predictor.predict_kfre(years=2, use_extra_vars=True, num_vars=8)
pred_8var_5year = predictor.predict_kfre(years=5, use_extra_vars=True, num_vars=8)


# inner join the different combinations of years and variables to
# kfre_inclusion dataframe
df_kfre = df_kfre.assign(
    pred_4var_2year=pred_4var_2year,
    pred_4var_5year=pred_4var_5year,
    pred_6var_2year=pred_6var_2year,
    pred_6var_5year=pred_6var_5year,
    pred_8var_2year=pred_8var_2year,
    pred_8var_5year=pred_8var_5year,
)

In [11]:
df_kfre

,Age,SEX,eGFR-EPI,uACR,Diabetes (1=yes; 0=no),Hypertension (1=yes; 0=no),Albumin (g/dL),Calcium (mmol/L),Phosphate (mmol/L),Bicarbonate (mmol/L),ESRD,Region,pred_4var_2year,pred_4var_5year,pred_6var_2year,pred_6var_5year,pred_8var_2year,pred_8var_5year
Patient_ID,,,,,,,,,,,,,,,,,,
568268916,87.24,Male,19.0,102.435547,1.0,1.0,3.7,2.78,0.88,27.2,0,North American,0.098693,0.277045,0.094041,0.265332,0.138546,0.384789
659549633,56.88,Female,15.0,1762.001840,0.0,1.0,3.0,2.43,1.02,21.3,1,North American,0.590171,0.938265,0.625652,0.953468,0.861472,0.998402
406748956,66.53,Female,17.0,659.122070,0.0,1.0,3.6,2.33,1.24,27.8,0,North American,0.309633,0.685513,0.335119,0.720360,0.408788,0.819502
109228713,69.92,Male,12.0,1145.220631,0.0,1.0,3.9,2.29,1.80,20.7,1,North American,0.626647,0.953853,0.662183,0.966230,0.866728,0.998591
220533110,81.14,Female,15.0,980.918742,1.0,1.0,4.3,2.45,1.39,26.2,0,North American,0.330726,0.714551,0.317278,0.696260,0.386981,0.796898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708268897,68.45,Female,23.0,3491.181746,0.0,1.0,2.4,2.38,0.53,27.8,0,North American,0.320402,0.700578,0.346535,0.735078,0.456723,0.862957
019128787,32.26,Female,16.0,3200.622314,0.0,1.0,4.1,2.46,1.51,21.3,1,North American,0.834012,0.996327,0.861675,0.997921,0.936959,0.999877
371958631,72.34,Male,20.0,3462.727326,1.0,1.0,4.0,2.51,0.96,22.9,0,North American,0.468142,0.860709,0.451243,0.846420,0.549579,0.925580


In [12]:
df_kfre["pred_4var_2year"].median()

0.32418652700752526